## Splitting whole files into 16 bar chunks

In [61]:
import os 

path_output = '/Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_drums/'
path_input = '/Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_dg-dg-b/'

## Assess there is an equal number of measures in each file

In [62]:
output = []
input = []

for file in os.listdir(path_output):
    output.append(os.path.join(path_output, file))

for file in os.listdir(path_input):
    input.append(os.path.join(path_input, file))

In [63]:
assert len(input) == len(output)

In [64]:
# Function to count occurrences of a string in a file
def count_occurrences_in_file(file_path, search_string):
    count = 0
    try:
        with open(file_path, 'r') as file:
            for line in file:
                count += line.count(search_string)
    except Exception as e:
        print("Could not process file {}: {}".format(file_path, e))
    return count

In [65]:
print(input[100])
print('\n')
print(output[100])

/Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_dg-dg-b/Obtest - Auka Seniems Dievams.gp4.tokens.txt


/Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_drums/Obtest - Auka Seniems Dievams.gp4.tokens.txt


In [66]:
search_string = 'new_measure'
exceptions = []

# Check if both drums and others have the same number of measures
for file in input:
    if count_occurrences_in_file(file, search_string) == count_occurrences_in_file(os.path.join(path_output, file.split('/')[-1]), search_string):
        continue
    else:
        print('Here is an exception: {}', file)
        exceptions.append(file)

In [67]:
import os

# Directory containing the text files
text_files_directory = path_input
output_directory = "/Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_16_dg-dg-b"

# Ensure the output directory exists
os.makedirs(output_directory, exist_ok=True)

# String to search for and the chunk size
search_string = "new_measure"
split_size = 16

# Define footer token
footer_token = "end\n"

# Function to split the file
def split_file(file_path, output_directory, search_string, split_size, footer_token):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()

        # Extract the first 4 lines as the header tokens
        header_tokens = lines[:4]

        # The remaining lines to process after the header
        remaining_lines = lines[4:]

        current_chunk = []
        occurrence_count = 0
        chunk_index = 0
        start_measure = 1
        carryover_line = None  # To hold the last discarded new_measure

        for line in remaining_lines:
            # If there is a carryover line from the previous chunk, add it first
            if carryover_line:
                current_chunk.append(carryover_line)
                carryover_line = None

            # Count occurrences of the search string in the current line
            if line.strip() == search_string:  # Ensure we only count full "new_measure" lines
                occurrence_count += 1

            current_chunk.append(line)

            # If the occurrence count reaches split_size + 1, save the chunk and handle carryover
            if occurrence_count == split_size + 1:
                # Remove the last "new_measure" token from the chunk and save it as carryover
                while current_chunk and current_chunk[-1].strip() != search_string:
                    current_chunk.pop()  # Remove non-"new_measure" lines at the end
                if current_chunk:
                    carryover_line = current_chunk.pop()  # Save the last "new_measure" as carryover

                # Calculate the range of measures for the current chunk
                end_measure = start_measure + split_size - 1

                # Save the chunk
                save_chunk(file_path, output_directory, current_chunk, chunk_index, start_measure, end_measure, header_tokens, footer_token)
                chunk_index += 1

                # Prepare for the next chunk
                current_chunk = []
                occurrence_count = 1  # The last "new_measure" starts the next chunk
                start_measure += split_size

        # Save any remaining content (if not empty)
        if current_chunk or carryover_line:
            # Add carryover line to the start of the final chunk
            if carryover_line:
                current_chunk.insert(0, carryover_line)

            # Calculate the final range of measures
            end_measure = start_measure + occurrence_count - 1

            # Check if footer token is already in the chunk
            if footer_token.strip() not in ''.join(current_chunk):
                current_chunk.append(footer_token)

            # Save the final chunk
            save_chunk(file_path, output_directory, current_chunk, chunk_index, start_measure, end_measure, header_tokens, footer_token)

    except Exception as e:
        print("Could not process file {}: {}".format(file_path, e))



# Function to save a chunk to a new file
def save_chunk(original_file_path, output_directory, chunk, chunk_index, start_measure, end_measure, header_tokens, footer_token):
    try:
        # Generate a new file name with the measure range
        base_name, file_extension = os.path.splitext(os.path.basename(original_file_path))
        measure_range = "_{}-{}_".format(start_measure, end_measure)
        if base_name.endswith(".tokens"):  # Handle files with ".tokens.txt" suffix
            base_name = base_name[:-7]  # Remove ".tokens" from the end of the base name
            new_file_name = "{}.{}.tokens{}".format(base_name, measure_range, file_extension)
        else:
            new_file_name = "{}{}{}".format(base_name, measure_range, file_extension)
        new_file_path = os.path.join(output_directory, new_file_name)

        with open(new_file_path, 'w') as new_file:
            # Write header tokens (first 4 lines of the original file)
            new_file.writelines(header_tokens)

            # Write the chunk content
            new_file.writelines(chunk)

            # Write the footer token
            new_file.write(footer_token + "\n")

        print("Saved chunk to {}".format(new_file_path))

    except Exception as e:
        print("Could not save chunk {}: {}".format(new_file_path, e))


In [68]:
split_file(input[0], output_directory, search_string, split_size, footer_token)

Saved chunk to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_16_dg-dg-b/HammerFall - Remember Yesterday (2).gp4._1-16_.tokens.txt
Saved chunk to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_16_dg-dg-b/HammerFall - Remember Yesterday (2).gp4._17-32_.tokens.txt
Saved chunk to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_16_dg-dg-b/HammerFall - Remember Yesterday (2).gp4._33-48_.tokens.txt
Saved chunk to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_16_dg-dg-b/HammerFall - Remember Yesterday (2).gp4._49-64_.tokens.txt
Saved chunk to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_16_dg-dg-b/HammerFall - Remember Yesterday (2).gp4._65-80_.tokens.txt
Saved chunk to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_16_dg-dg-b/HammerFall - Remember Yesterday (2).gp4._81-96_.tokens.txt
Saved chunk to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_16_dg-dg-b/HammerFall - Remember Yesterday (2).gp

All the tracks seem to have the same number of bars, as expected.

In [71]:
from tqdm import tqdm
# Process each file in the directory
for file in tqdm(input):
    print(file)
    if file.endswith(".txt"):
        split_file(file, output_directory, search_string, split_size, footer_token)


  0%|          | 17/7636 [00:00<00:45, 168.80it/s]

/Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_dg-dg-b/Scorpions - The Zoo (2).gp4.tokens.txt
Saved chunk to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_16_dg-dg-b/Scorpions - The Zoo (2).gp4._1-16_.tokens.txt
Saved chunk to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_16_dg-dg-b/Scorpions - The Zoo (2).gp4._17-32_.tokens.txt
Saved chunk to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_16_dg-dg-b/Scorpions - The Zoo (2).gp4._33-48_.tokens.txt
Saved chunk to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_16_dg-dg-b/Scorpions - The Zoo (2).gp4._49-64_.tokens.txt
Saved chunk to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_16_dg-dg-b/Scorpions - The Zoo (2).gp4._65-80_.tokens.txt
Saved chunk to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_16_dg-dg-b/Scorpions - The Zoo (2).gp4._81-96_.tokens.txt
Saved chunk to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_16_dg-dg-b/Sc

  1%|          | 56/7636 [00:00<00:40, 186.25it/s]

Saved chunk to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_16_dg-dg-b/Scorpions - The Zoo (2).gp4._49-64_.tokens.txt
Saved chunk to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_16_dg-dg-b/Scorpions - The Zoo (2).gp4._65-80_.tokens.txt
Saved chunk to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_16_dg-dg-b/Scorpions - The Zoo (2).gp4._81-96_.tokens.txt
Saved chunk to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_16_dg-dg-b/Scorpions - The Zoo (2).gp4._97-112_.tokens.txt
Saved chunk to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_16_dg-dg-b/Scorpions - The Zoo (2).gp4._113-128_.tokens.txt
Saved chunk to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_16_dg-dg-b/Scorpions - The Zoo (2).gp4._129-139_.tokens.txt
/Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_dg-dg-b/Scorpions - The Zoo (2).gp4.tokens.txt
Saved chunk to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_16_dg-d

  1%|          | 95/7636 [00:00<00:39, 190.65it/s]

...

 20%|█▉        | 1522/7636 [00:07<00:34, 179.57it/s]

Saved chunk to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_16_dg-dg-b/Scorpions - The Zoo (2).gp4._17-32_.tokens.txt
Saved chunk to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_16_dg-dg-b/Scorpions - The Zoo (2).gp4._33-48_.tokens.txt
Saved chunk to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_16_dg-dg-b/Scorpions - The Zoo (2).gp4._49-64_.tokens.txt
Saved chunk to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_16_dg-dg-b/Scorpions - The Zoo (2).gp4._65-80_.tokens.txt
Saved chunk to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_16_dg-dg-b/Scorpions - The Zoo (2).gp4._81-96_.tokens.txt
Saved chunk to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_16_dg-dg-b/Scorpions - The Zoo (2).gp4._97-112_.tokens.txt
Saved chunk to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_16_dg-dg-b/Scorpions - The Zoo (2).gp4._113-128_.tokens.txt
Saved chunk to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-a

 21%|██        | 1569/7636 [00:08<00:31, 190.77it/s]

Saved chunk to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_16_dg-dg-b/Scorpions - The Zoo (2).gp4._113-128_.tokens.txt
Saved chunk to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_16_dg-dg-b/Scorpions - The Zoo (2).gp4._129-139_.tokens.txt
/Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_dg-dg-b/Scorpions - The Zoo (2).gp4.tokens.txt
Saved chunk to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_16_dg-dg-b/Scorpions - The Zoo (2).gp4._1-16_.tokens.txt
Saved chunk to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_16_dg-dg-b/Scorpions - The Zoo (2).gp4._17-32_.tokens.txt
Saved chunk to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_16_dg-dg-b/Scorpions - The Zoo (2).gp4._33-48_.tokens.txt
Saved chunk to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_16_dg-dg-b/Scorpions - The Zoo (2).gp4._49-64_.tokens.txt
Saved chunk to /Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/data/txts_16_dg-dg-

KeyboardInterrupt: 